# Add Patient Transcripts

This notebook adds existing synthetic patient transcripts to our Postgres database to be used in test cases.

In [28]:
import os
import json
import random
import string

# point to either DEV (default) or PROD
# os.environ["POSTGRES_DB_ENVIRONMENT"] = "PROD"

from reco_analysis.data_model import data_models
from reco_analysis.chatbot import chatbot
from langchain.schema import BaseChatMessageHistory, AIMessage, HumanMessage


session = data_models.get_session()

transcripts_version = "1.0"
transcripts_json_file_path = f"../../data/patients/patients_{transcripts_version}_with_transcripts.json"

with open(transcripts_json_file_path, 'r') as json_file:
    transcripts = json.load(json_file)

In [27]:
default_hcp = data_models.HealthcareProvider.get_by_id(1, session)

for transcript in transcripts.values():
    patient_name: str = transcript["name"]
    first_name, last_name = patient_name.split(" ")
    username = patient_name.lower().replace(" ", "_")
    password = ''.join(random.choices(string.ascii_letters + string.digits, k=20))

    # create patient
    try:
        patient = data_models.Patient(
            username=username,
            first_name=first_name,
            last_name=last_name,
            email=username + "@example.com",
            password=password,
        )
        patient.healthcare_provider = default_hcp
        session.add(patient)
        session.flush()
        print(f"Created patient {patient_name}, id: {patient.id}")
    except Exception as e:
        print(f"Error creating patient {patient_name}: {e}")
        session.rollback()
        continue

    # create conversation session
    conversation_session = patient.new_session()
    session.add(conversation_session)
    session.commit()
    print(f"Created conversation session for {patient_name}, id: {conversation_session.id}")

    message_history: BaseChatMessageHistory = chatbot.get_session_history(conversation_session.id)
    print(f"Adding messages to conversation session for {patient_name} at {conversation_session.id}")
    for message in transcript["chat_transcript"]:
        # either "Doctor: " or "Patient: "
        role, content = message.split(": ", 1)
        message_class = {"Doctor": AIMessage, "Patient": HumanMessage}[role]
        message_history.add_message(message_class(content=content, name=role))

    conversation_session.completed = True
    session.commit()

Error creating patient Kevin Morris: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "patients_email_key"
DETAIL:  Key (email)=(kevin_morris@example.com) already exists.

[SQL: INSERT INTO patients (username, first_name, last_name, email, password, healthcare_provider_id) VALUES (%(username)s, %(first_name)s, %(last_name)s, %(email)s, %(password)s, %(healthcare_provider_id)s) RETURNING patients.id, patients.created_at, patients.updated_at]
[parameters: {'username': 'kevin_morris', 'first_name': 'Kevin', 'last_name': 'Morris', 'email': 'kevin_morris@example.com', 'password': 'lI7s9jzvB2Ex3d7VD7M9', 'healthcare_provider_id': 1}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Created patient Michael Owens, id: 13
Created conversation session for Michael Owens, id: 2fa4d8d0-e5ad-4342-b80e-f118aec0bbe4
Adding messages to conversation session for Michael Owens at 2fa4d8d0-e5ad-4342-b80e-f118aec0bbe4
Created patient Robert Henderson, id: 14
Creat